In [17]:
import tushare as ts
import talib as ta
import os,time,sys,re,datetime
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
import datetime as dt
import pymysql

import pyodbc

In [18]:
df = ts.get_today_all()

[Getting data:]############################################################

In [29]:
df.head(100)
a = list(df['code'])
df1 = pd.read_csv("data/电信运营2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

df2 = pd.read_csv("data/酒店餐饮2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

df3 = pd.concat([df1, df2]) #上下合并两个df

codes = list(df3['code'])
for code in codes:
    code_str = str(code).zfill(6)
    df4 = ts.get_hist_data(code_str)


002467
            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-12-03  5.17  5.25   5.25  5.10  148002.73          0.06      1.16  5.218   
2019-12-02  5.20  5.23   5.19  5.17  112877.91          0.00      0.00  5.226   
2019-11-29  5.22  5.22   5.19  5.15  126164.84         -0.02     -0.38  5.242   
2019-11-28  5.27  5.27   5.21  5.20  138587.20         -0.04     -0.76  5.272   
2019-11-27  5.29  5.30   5.25  5.23  129196.56         -0.04     -0.76  5.324   

             ma10   ma20      v_ma5     v_ma10     v_ma20  
date                                                       
2019-12-03  5.298  5.352  130965.85  221563.13  208683.56  
2019-12-02  5.313  5.370  124369.29  231665.17  211517.15  
2019-11-29  5.323  5.392  142082.79  240901.05  218342.48  
2019-11-28  5.341  5.417  175398.28  255528.79  223641.24  
2019-11-27  5.350  5.439  212873.18  255550.63  234081.02  
60388

            open  high  close   low    volume  price_change  p_change    ma5  \
date                                                                           
2019-12-03  2.50  2.62   2.58  2.48  29669.00          0.04      1.57  2.482   
2019-12-02  2.44  2.56   2.54  2.44  32902.00          0.10      4.10  2.464   
2019-11-29  2.39  2.44   2.44  2.39  12608.00          0.03      1.25  2.470   
2019-11-28  2.42  2.45   2.41  2.40  17725.00         -0.03     -1.23  2.472   
2019-11-27  2.49  2.49   2.44  2.41  26993.53         -0.05     -2.01  2.470   

             ma10   ma20     v_ma5    v_ma10    v_ma20  
date                                                    
2019-12-03  2.477  2.386  23979.51  34102.64  29801.19  
2019-12-02  2.452  2.382  32539.71  34168.58  29534.49  
2019-11-29  2.420  2.381  33390.91  32306.88  30323.20  
2019-11-28  2.397  2.389  35558.51  32281.38  30475.15  
2019-11-27  2.378  2.400  38612.55  32256.28  30234.60  
600754
             open   high  close  

In [9]:
DBfile = r".\data\stock.mdb"  # 数据库文件  
try:
    conn = pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\data\stock.mdb')
    cur = conn.cursor()
except Exception as e:
     print('engine err!')

In [13]:
# 1)获得更新的所有标的数据
def getstock():
    df = ts.get_today_all()   #获取当日标的所有交易数据
    df = df[['code', 'name', 'open', 'high', 'trade', 'low', 'volume', 'amount']]
    df = df.set_index('code')
    df = df.sort_index()
    
    return df

# 2）、获取单只标的交易数据
def Get_Stock_Trade(code, conn):
    try:
        #dataframe里的date在数据库里是trade_date
        sql = "SELECT trade_date,low,volume FROM _trade where code = '" + code + "'"
        #sql = "select * from _trade where code = '002467'"
        #print(sql)
        df = pd.read_sql(sql, conn)
        #print(df.head(100))
        df = df.set_index('trade_date')
        df = df.sort_index()

    except Exception as e:
        print(code, 'sql err!')
        print(e)
    
    return df


# 3)获取构造均价、均量、MACD数据
def Get_Stock_ma_vol(df_Code,date=None):
    print("starting.....")
    sql_list=[]
    no_arreay = []
    slen=df_Code.shape[0]
    i = 1

    for code,row in df_Code.iterrows():
        open,high,low,close,volume,amount = row[1:7]
        print(str(i)+"/"+str(slen)+","+code),
        i = i + 1

        try:
            df = Get_Stock_Trade(code, conn)  #获取标的之前的交易数据，以便于今日数据合并
            # your codes ，发现标的无记录，有的code在你的数据库表中无记录
             
                

        except Exception as e:
            print(code + 'trade err!')
            print(e)
            continue

        dflen = df.shape[0]

        #在历史交易df中加入今日的新记录
        if date is None:
            TODAY = dt.date.today()
            date=TODAY.strftime('%Y-%m-%d')
            
        new = pd.DataFrame('your codes')
        df=df.append(new)
        
        # your codes  构造成交价和成交量均线
        
        
        # 构造成交价和成交量均线,根据具体情况构造n日均价，n日均线
        dfma25 = df['close'].rolling(min_periods=1, window=25, center=False).mean()   # 25日均价
        dfvol60 =df['volume'].rolling(min_periods=1, window=60, center=False).mean()  # 60日均量

        
        # 构造MACD值，psmacd DIFF；psmacdsignal DEA；psmacdhist DIFF-DEA
        # http://www.lfd.uci.edu/~gohlke/pythonlibs/ 下载安装 TA-Lib
        psmacd, psmacdsignal, psmacdhist = ta.MACD(np.array(df['close']), fastperiod=12, slowperiod=26, signalperiod=9)
        #转换为series
        

      
        # your code 构造MACD


        #在后面增加5列，分别是14-18列，对应的是 ma25 vol60 DIFF  DEA  DIFF-DEA       
        df['ma25']=pd.Series(dfma25,index=df.index)             #ma25
        df['v_ma60']=pd.Series(dfvol60,index=df.index)          #vol60
        df['macd']=pd.Series(psmacd,index=df.index)             #DIFF
        df['macdsignal']=pd.Series(psmacdsignal,index=df.index) #DEA
        df['macdhist']=pd.Series(psmacdhist,index=df.index)     #DIFF-DEA
        
        
        sqls = []
        
        #print(code, date, open, high, low, close, volume, amount, macd, macdsignal, macdhist,ma5,ma25,vol5,vol60)
        sql = "insert into _trade(code,date,open,high,close,low,volume,amount,macd,macdsignal,macdhist,"\
        "ma5,ma25,vol5,vol60) values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%d','%.3f','%.3f',"\
"'%.3f','%.2f','%.2f','%d','%d')" % (code, date, open, high, low, close, volume, amount, macd, macdsignal, macdhist,ma5,ma25,vol5,vol60)
        
        sqls.append(sql)

        Insert_Stock_Trade(sqls, engine) ##追加数据库
        
    conn.close()
    print("End!")
    return no_arreay

def Insert_Stock_Trade(sqls, engine):
    try:
        for insert_sql in sqls:
            engine.execute(insert_sql)
            #print(insert_sql)
    except Exception as e:
        print(e)
        print('sql err!')

    return df


In [14]:
#调用各函数，完成每日数据更新
#your codes
df1 = Get_Stock_Trade('002467',conn)
Get_Stock_ma_vol(df1)


starting.....


ValueError: not enough values to unpack (expected 6, got 1)

### 任务列表
1、利用MACD数据判断底背离与顶背离，_trade表中最好包含所有标的的交易数据<br>
2、从_trade表中发现5日均量大于60日均量，25日均价线保持上升的标的<br>
3、利用 _trade 表数据制作透视表，使用数据框的pivot_table函数<br>
4、{一维Series，二维DataFrame，三维Panel，Panel中的每一项（类似于DataFrame的列）都是一个DataFrame}，利用 _trade表进行面板数据分析<br>
5、利用 _trade 表进行分组groupby（），交叉表crosstab的应用<br>
6、pandas 统计函数的应用<br>

count               非NA值的数量
describe            针对Series或各DataFrame列计算汇总统计
min,max             计算最小值和最大值
argmin,argmax       计算能够获取到最小值和最大值的索引位置（整数)
idxmin,idxmax       计算能够获取到最小值和最大值的索引值
quantile            计算样本的分位数（0到 1）
sum                 值的总和
mean                值的平均数， a.mean() 默认对每一列的数据求平均值；若加上参数a.mean(1)则对每一行求平均值
media               值的算术中位数（50%分位数)
mad                 根据平均值计算平均绝对离差
var                 样本值的方差
std                 样本值的标准差
skew                样本值的偏度（三阶矩）
kurt                样本值的峰度（四阶矩）
cumsum              样本值的累计和
cummin,cummax       样本值的累计最大值和累计最小
cumprod             样本值的累计积
diff                计算一阶差分（对时间序列很有用)
pct_change          计算百分数变化

### 透视表应用示例

In [15]:
#透视表  通过两张表_trade,_basic
date='2017-02-20'
sql = "SELECT A.date,A.code,A.low,A.volume,B.industry,B.area FROM _trade A,_basic B where A.code=B.code and A.date>='" + date + "'"
df = pd.read_sql(sql, conn)
df = df.set_index('date')
df = df.sort_index()

DatabaseError: Execution failed on sql 'SELECT A.date,A.code,A.low,A.volume,B.industry,B.area FROM _trade A,_basic B where A.code=B.code and A.date>='2017-02-20'': ('42S02', "[42S02] [Microsoft][ODBC Microsoft Access Driver] The Microsoft Access database engine cannot find the input table or query '_basic'. Make sure it exists and that its name is spelled correctly. (-1305) (SQLExecDirectW)")

In [34]:
df.head()

,code,low,volume,industry,area
date,,,,,
2017-02-20,000001,9.56,89875539,银行,深圳
2017-02-20,000002,20.61,24748349,全国地产,深圳
2017-02-20,000004,37.46,1127449,生物制药,深圳
2017-02-20,000005,6.61,7987002,房产服务,深圳
2017-02-20,000006,8.84,12995962,区域地产,深圳


In [35]:
tb=pd.pivot_table(df,values='volume',index=['industry'],columns=['area'], aggfunc=np.sum)

In [50]:
tb

area,上海,云南,内蒙,北京,吉林,四川,天津,宁夏,安徽,山东,...,湖南,甘肃,福建,西藏,贵州,辽宁,重庆,陕西,青海,黑龙江
industry,,,,,,,,,,,,,,,,,,,,,
专用机械,49439089.0,NaN,NaN,2.345586e+08,NaN,189641173.0,17943963.0,NaN,7.744572e+07,44493276.0,...,NaN,NaN,6.788408e+07,NaN,NaN,336236562.0,NaN,47262234.0,NaN,NaN
中成药,81022441.0,96586865.0,10387816.0,6.735624e+07,160950288.0,30010893.0,189154140.0,NaN,NaN,68107371.0,...,67959811.0,98369072.0,2.200905e+07,22033457.0,251746431.0,NaN,33795907.0,24501876.0,6077396.0,35526690.0
乳制品,45981286.0,NaN,333278029.0,2.526822e+07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
互联网,282625818.0,NaN,NaN,3.835616e+08,NaN,NaN,NaN,NaN,3.998600e+07,83624532.0,...,44352962.0,NaN,6.032004e+07,NaN,NaN,40807342.0,NaN,NaN,NaN,NaN
仓储物流,85226491.0,NaN,NaN,3.929309e+07,NaN,NaN,120503735.0,NaN,6.775816e+07,18029533.0,...,NaN,NaN,4.188892e+07,NaN,NaN,75486174.0,NaN,NaN,NaN,12978939.0
供气供热,110827783.0,NaN,NaN,NaN,432341470.0,31318885.0,11297124.0,NaN,NaN,110854142.0,...,NaN,NaN,NaN,NaN,NaN,112622790.0,31308770.0,34587635.0,NaN,NaN
保险,88053205.0,NaN,57733967.0,9.742281e+07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
元器件,93247424.0,NaN,NaN,1.780833e+09,NaN,148948973.0,NaN,NaN,1.003816e+08,206672331.0,...,72472636.0,NaN,2.908234e+08,NaN,22679485.0,NaN,105167297.0,46901847.0,NaN,NaN
全国地产,461145297.0,119552812.0,NaN,2.150061e+08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.140586e+07,NaN,NaN,46904533.0,200771895.0,NaN,NaN,NaN
